## Updating columns

In [ ]:
# Import modules

import json
from http import HTTPStatus

import requests

In [ ]:
# Enter your access token, Evo hub URL, org ID, workspace ID, and block model ID.

access_token = ""
evo_hub_url = ""
org_id = ""
workspace_id = ""
block_model_id = ""

auth_header = {"Authorization": "Bearer " + access_token}

In [ ]:
# Compose the URL and JSON body
url = f"{evo_hub_url}/blockmodel/orgs/{org_id}/workspaces/{workspace_id}/block-models/{block_model_id}/blocks"

body = {
    "columns": {"new": [], "delete": [], "update": ["Geology"], "rename": []},
    "comment": "Comment updated during UPDATE column.",
}

# Make a PATCH request and include the JSON body and headers
response = requests.patch(url=url, json=body, headers=auth_header)
response_output = json.dumps(response.json(), indent=4, sort_keys=True)

# Copy the `upload_url` and `job_url` parameters from the response
if response.status_code == HTTPStatus.ACCEPTED:
    print(response_output)
    upload_url = response.json()["upload_url"]
    job_url = response.json()["job_url"]
else:
    raise Exception(f"Request failed: \n Status: {response.status_code} \n Response: {response_output}")

In [ ]:
# Compose the headers
# NOTE: The Seequent ID access token is not required for this request
headers = {"Content-Type": "application/binary", "x-ms-blob-type": "BlockBlob"}

# Make a PUT request - include the binary data and headers
with open("sample/data_update.parquet", "rb") as data_stream:
    response = requests.put(url=upload_url, data=data_stream, headers=headers)

if response.status_code != HTTPStatus.CREATED:
    raise Exception(f"Request failed: \n Status: {response.status_code} \n Response: {response.json()}")

In [ ]:
# Compose the URL
url = f"{job_url}/uploaded"

# Make a POST request, including the headers
response = requests.post(url=url, headers=auth_header)
response_output = json.dumps(response.json(), indent=4, sort_keys=True)
print(response_output)

In [ ]:
# Keep checking the `job_url` until the `job_status` is `COMPLETE`.

# Make a GET request, including the headers
response = requests.get(url=job_url, headers=auth_header)
response_output = json.dumps(response.json(), indent=4, sort_keys=True)
print(response_output)